# DX 704 Week 8 Project

This homework will modify a simulator controlling a small vehicle to implement tabular q-learning.
You will first test your code with random and greedy-epsilon policies, then tweak your own training method for a more optimal policy.

The full project description and a template notebook are available on GitHub: [Project 8 Materials](https://github.com/bu-cds-dx704/dx704-project-08).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Rover Simulator

The following Python class implements a simulation of a simple vehicle with integer x,y coordinates facing in one of 8 possible directions.


In [3]:
# DO NOT CHANGE

import random

class RoverSimulator(object):
    DIRECTIONS = ((0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1))

    def __init__(self, resolution):
        self.resolution = resolution
        self.terminal_state = self.construct_state(resolution // 2, resolution // 2, 0)

        self.initial_states = []
        for initial_x in (0, resolution // 2, resolution - 1):
            for initial_y in (0, resolution // 2, resolution - 1):
                for initial_direction in range(8):
                    initial_state = self.construct_state(initial_x, initial_y, initial_direction)
                    if initial_state != self.terminal_state:
                        self.initial_states.append(initial_state)

    def construct_state(self, x, y, direction):
        assert 0 <= x < self.resolution
        assert 0 <= y < self.resolution
        assert 0 <= direction < 8

        state = (y * self.resolution + x) * 8 + direction
        assert self.decode_state(state) == (x, y, direction)
        return state

    def decode_state(self, state):
        direction = state % 8
        x = (state // 8) % self.resolution
        y = state // (8 * self.resolution)

        return (x, y, direction)

    def get_actions(self, state):
        return [-1, 0, 1]

    def get_next_reward_state(self, curr_state, curr_action):
        if curr_state == self.terminal_state:
            # no rewards or changes from terminal state
            return (0, curr_state)

        (curr_x, curr_y, curr_direction) = self.decode_state(curr_state)
        (curr_dx, curr_dy) = self.DIRECTIONS[curr_direction]

        assert self.construct_state(curr_x, curr_y, curr_direction) == curr_state

        assert curr_action in (-1, 0, 1)

        next_x = min(max(0, curr_x + curr_dx), self.resolution - 1)
        next_y = min(max(0, curr_y + curr_dy), self.resolution - 1)
        next_direction = (curr_direction + curr_action) % 8

        next_state = self.construct_state(next_x, next_y, next_direction)
        next_reward = 1 if next_state == self.terminal_state else 0

        return (next_reward, next_state)

    def rollout_policy(self, policy_func, max_steps=1000):
        curr_state = self.sample_initial_state()
        for i in range(max_steps):
            curr_action = policy_func(curr_state, self.get_actions(curr_state))
            (next_reward, next_state) = self.get_next_reward_state(curr_state, curr_action)
            yield (curr_state, curr_action, next_reward, next_state)
            curr_state = next_state

    def sample_initial_state(self):
        return random.choice(self.initial_states)

In [4]:
simulator = RoverSimulator(16)
initial_sample = simulator.sample_initial_state()
print("INITIAL SAMPLE", initial_sample)

INITIAL SAMPLE 66


## Part 1: Implement a Random Policy

Random policies are often used to test simulators and start initial exploration.
Implement a random policy for these simulators.

In [5]:
# YOUR CHANGES HERE

def random_policy(state, actions):
    import random
    return random.choice(actions)

with open("log-random.tsv", "w") as fh:
    fh.write("curr_state\tcurr_action\tnext_reward\tnext_state\n")
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
        fh.write(f"{curr_state}\t{curr_action}\t{next_reward}\t{next_state}\n")
        print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)

CURR STATE 1144 ACTION 1 NEXT REWARD 0 NEXT STATE 1273
CURR STATE 1273 ACTION 0 NEXT REWARD 0 NEXT STATE 1401
CURR STATE 1401 ACTION -1 NEXT REWARD 0 NEXT STATE 1528
CURR STATE 1528 ACTION 1 NEXT REWARD 0 NEXT STATE 1657
CURR STATE 1657 ACTION -1 NEXT REWARD 0 NEXT STATE 1784
CURR STATE 1784 ACTION 1 NEXT REWARD 0 NEXT STATE 1913
CURR STATE 1913 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 1 NEXT REWARD 0 NEXT STATE 2042
CURR STATE 2042 ACTION -1 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 1 NEXT REWARD 0 NEXT STATE 2042
CURR STATE 2042 ACTION 0 NEXT REWARD 0 NEXT STATE 2042
CURR STATE 2042 ACTION 1 NEXT REWARD 0 NEXT STATE 2043
CURR STATE 2043 ACTION 1 NEXT REWARD 0 NEXT STATE 1916
CURR STATE 1916 ACTION -1 NEXT REWARD 0 NEXT STATE 1787
CURR STATE 1787 ACTION 0 NEXT REWARD 0 NEXT STATE 1659
CURR STATE 1659 ACTION -1 NEXT REWARD 0 NEXT STATE 1530
CURR STATE 1530 ACTION -1 NEXT REWARD 0 NEXT STATE 1529
CURR

Use the code below to test your random policy.
Then modify it to save the results in "log-random.tsv" with the columns curr_state, curr_action, next_reward and next_state.

In [6]:
# YOUR CHANGES HERE

for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
    print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)

CURR STATE 2044 ACTION -1 NEXT REWARD 0 NEXT STATE 1915
CURR STATE 1915 ACTION 0 NEXT REWARD 0 NEXT STATE 1787
CURR STATE 1787 ACTION 0 NEXT REWARD 0 NEXT STATE 1659
CURR STATE 1659 ACTION -1 NEXT REWARD 0 NEXT STATE 1530
CURR STATE 1530 ACTION 0 NEXT REWARD 0 NEXT STATE 1530
CURR STATE 1530 ACTION 0 NEXT REWARD 0 NEXT STATE 1530
CURR STATE 1530 ACTION -1 NEXT REWARD 0 NEXT STATE 1529
CURR STATE 1529 ACTION -1 NEXT REWARD 0 NEXT STATE 1656
CURR STATE 1656 ACTION -1 NEXT REWARD 0 NEXT STATE 1791
CURR STATE 1791 ACTION -1 NEXT REWARD 0 NEXT STATE 1910
CURR STATE 1910 ACTION -1 NEXT REWARD 0 NEXT STATE 1901
CURR STATE 1901 ACTION 0 NEXT REWARD 0 NEXT STATE 1765
CURR STATE 1765 ACTION 0 NEXT REWARD 0 NEXT STATE 1629
CURR STATE 1629 ACTION 0 NEXT REWARD 0 NEXT STATE 1493
CURR STATE 1493 ACTION 1 NEXT REWARD 0 NEXT STATE 1358
CURR STATE 1358 ACTION 1 NEXT REWARD 0 NEXT STATE 1351
CURR STATE 1351 ACTION 0 NEXT REWARD 0 NEXT STATE 1471
CURR STATE 1471 ACTION -1 NEXT REWARD 0 NEXT STATE 1590
CU

Submit "log-random.tsv" in Gradescope.

## Part 2: Implement Q-Learning with Random Policy

The code below runs 32 random rollouts of 1024 steps using your random policy.
Modify the rollout code to implement Q-Learning.
Just implement one learning update for each sampled state-action in the simulation.
Use $\alpha=1$ and $\gamma=0.9$ since the simulator is deterministic and there is a sink where the rewards stop.




In [7]:
# YOUR CHANGES HERE

Q = {}
alpha = 1.0
gamma = 0.9
rows_random = []

for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=1024):
        old_value = Q.get((curr_state, curr_action), 0.0)
        next_actions = simulator.get_actions(next_state)
        max_next = max((Q.get((next_state, a), 0.0) for a in next_actions), default=0.0)

        new_value = (1 - alpha) * old_value + alpha * (next_reward + gamma * max_next)
        Q[(curr_state, curr_action)] = new_value

        rows_random.append((curr_state, curr_action, next_reward, next_state, old_value, new_value))



Save each step in the simulator in a file "q-random.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

In [8]:
# YOUR CHANGES HERE
with open("q-random.tsv", "w") as fh:
    fh.write("curr_state\tcurr_action\tnext_reward\tnext_state\told_value\tnew_value\n")
    for (cs, ca, nr, ns, ov, nv) in rows_random:
        fh.write(f"{cs}\t{ca}\t{nr}\t{ns}\t{ov}\t{nv}\n")
...

Ellipsis

Submit "q-random.tsv" in Gradescope.

## Part 3: Implement Epsilon-Greedy Policy

Implement an epsilon-greedy policy that picks the optimal policy based on your q-values so far 75% of the time, and picks a random action 25% of the time.
This is a high epsilon value, but the environment is deterministic, so it will benefit from more exploration.

In [9]:
Q = {}                     
alpha = 1.0
gamma = 0.9
rows_greedy = []


def epsilon_greedy_policy(state, actions):
    if random.random() < 0.25:
        return random.choice(actions)
    qvals = [Q.get((state, a), 0.0) for a in actions]
    max_q = max(qvals)
    best_actions = [a for a, q in zip(actions, qvals) if q == max_q]
    return random.choice(best_actions)



Combine your epsilon-greedy policy with q-learning below and save the observations and updates in "q-greedy.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

Hint: make sure to reset your q-learning state before running the simulation below so that the learning process is recorded from the beginning.

In [10]:
# YOUR CHANGES HERE

for episode in range(32):
    curr_state = simulator.sample_initial_state()
    for step in range(1024):
        curr_action = epsilon_greedy_policy(curr_state, simulator.get_actions(curr_state))
        (next_reward, next_state) = simulator.get_next_reward_state(curr_state, curr_action)

        old_value = Q.get((curr_state, curr_action), 0.0)
        next_actions = simulator.get_actions(next_state)
        max_next = max((Q.get((next_state, a), 0.0) for a in next_actions), default=0.0)

        new_value = (1 - alpha) * old_value + alpha * (next_reward + gamma * max_next)
        Q[(curr_state, curr_action)] = new_value

        rows_greedy.append((curr_state, curr_action, next_reward, next_state, old_value, new_value))

        curr_state = next_state

with open("q-greedy.tsv", "w") as fh:
    fh.write("curr_state\tcurr_action\tnext_reward\tnext_state\told_value\tnew_value\n")
    for (cs, ca, nr, ns, ov, nv) in rows_greedy:
        fh.write(f"{cs}\t{ca}\t{nr}\t{ns}\t{ov}\t{nv}\n")



Submit "q-greedy.tsv" in Gradescope.

## Part 4: Extract Policy from Q-Values

Using your final q-values from the previous simulation, extract a policy picking the best actions according to those q-values.
Save the policy in a file "policy-greedy.tsv" with columns state and action.

In [11]:
# YOUR CHANGES HERE
num_states = simulator.resolution * simulator.resolution * 8

with open("policy-greedy.tsv", "w") as fh:
    fh.write("state\taction\n")
    for state in range(num_states):
        actions = simulator.get_actions(state)
        best_action = max(actions, key=lambda a: Q.get((state, a), 0.0))
        fh.write(f"{state}\t{best_action}\n")
...

Ellipsis

Submit "policy-greedy.tsv" in Gradescope.

## Part 5: Implement Large Policy

Train a more optimal policy using q-learning.
Save the policy in a file "policy-optimal.tsv" with columns state and action.

Hint: this policy will be graded on its performance compared to optimal for each of the initial states.
**You will get full credit if the average value of your policy for the initial states is within 20% of optimal.**
Make sure that your policy has coverage of all the initial states, and does not take actions leading to states not included in your policy.
You will have to run several rollouts to get coverage of all the initial states, and the provided loops for parts 2 and 3 only consist of one rollout each.

Hint: this environment only gives one non-zero reward per episode, so you may want to cut off rollouts for speed once they get that reward.
But make sure you update the q-values first!

In [12]:
Q = {}
alpha = 1.0
gamma = 0.9
import random

def train_q(episodes=2000, epsilon=0.25, max_steps=1024):
    for ep in range(episodes):
        def eps_policy(state, actions):
            if random.random() < epsilon:
                return random.choice(actions)
            qvals = [Q.get((state, a), 0.0) for a in actions]
            max_q = max(qvals)
            best_actions = [a for a, q in zip(actions, qvals) if q == max_q]
            return random.choice(best_actions)

        for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(eps_policy, max_steps=max_steps):
            old_value = Q.get((curr_state, curr_action), 0.0)
            next_actions = simulator.get_actions(next_state)
            max_next = max((Q.get((next_state, a), 0.0) for a in next_actions), default=0.0)
            new_value = (1 - alpha) * old_value + alpha * (next_reward + gamma * max_next)
            Q[(curr_state, curr_action)] = new_value
            # intentionally do NOT break on next_reward so each episode yields max_steps entries

train_q(episodes=2000, epsilon=0.25, max_steps=1024)

num_states = simulator.resolution * simulator.resolution * 8
with open("policy-optimal.tsv", "w") as fh:
    fh.write("state\taction\n")
    for state in range(num_states):
        actions = simulator.get_actions(state)
        best_action = max(actions, key=lambda a: Q.get((state, a), 0.0))
        fh.write(f"{state}\t{best_action}\n")

Submit "policy-optimal.tsv" in Gradescope.

In [13]:
import pandas as pd

df_random = pd.read_csv("q-greedy.tsv", sep="\t")  
df_random.shape 

(32768, 6)

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.